In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
# Load the dataset we'll use for this lab
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True, data_dir='./data')

mnist_train, mnist_test = datasets['train'], datasets['test']

Dataset mnist downloaded and prepared to ./data\mnist\3.0.1. Subsequent calls will reuse this data.


In [3]:
# Define the strategy to use and print the number of devices found
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Number of devices: 1


In [4]:
# Get the number of examples in the train and test sets
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
# Use for Mirrored Strategy
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
# Use for No Strategy
# BATCH_SIZE = BATCH_SIZE_PER_REPLICA * 1

In [5]:
# Function for normalizing the image
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

In [6]:
# Set up the train and eval data set
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [7]:
# Use for Mirrored Strategy -- comment out `with strategy.scope():` and deindent for no strategy
with strategy.scope():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
    ])

In [8]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [9]:
model.fit(train_dataset, epochs=12)

Epoch 1/12
938/938 [==============================] - 38s 32ms/step - loss: 0.4031 - accuracy: 0.8849
Epoch 2/12
938/938 [==============================] - 22s 23ms/step - loss: 0.0679 - accuracy: 0.9801
Epoch 3/12
938/938 [==============================] - 23s 24ms/step - loss: 0.0482 - accuracy: 0.9855
Epoch 4/12
938/938 [==============================] - 27s 28ms/step - loss: 0.0338 - accuracy: 0.9901
Epoch 5/12
938/938 [==============================] - 28s 30ms/step - loss: 0.0269 - accuracy: 0.9921
Epoch 6/12
938/938 [==============================] - 25s 26ms/step - loss: 0.0190 - accuracy: 0.99431s - loss: 0.0 - ETA: 0s - l
Epoch 7/12
938/938 [==============================] - 27s 29ms/step - loss: 0.0145 - accuracy: 0.9952
Epoch 8/12
938/938 [==============================] - 26s 28ms/step - loss: 0.0107 - accuracy: 0.9967
Epoch 9/12
938/938 [==============================] - 24s 26ms/step - loss: 0.0104 - accuracy: 0.9962
Epoch 10/12
938/938 [==============================] -